# PS 6: Particle Tracking

#### Description:

Recall that we need a variety of elements:
1. model - this is the “thing” that is actually being tracked. Maybe it’s a patch, a contour, or some other description of an entity
2. a representation of state xt that describes the state of the model at time t;
3. a dynamics model p(xt | xt-1)  that describes the distribution of the state at time t given the state at t − 1;
4. a measurement zt that some how captures the data of the current image; and finally,
5. a sensor model p(zt | xt) that gives the likelihood of a measurement given the state. For Bayesian-based tracking, we assume that at time t − 1 we have a belief about the state represented as a density p(xt-1),  and that given some measurements zt at time t we update our Belief by computing the posterior density:
Bel(xt) ∝ p(zt | xt)p(xt | ut, xt-1)Bel(xt-1)

The Kalman filter provided an analytic method for doing this under the assumption that density representing the belief at time t, the noise component of the dynamics and the sensor model were all simple Gaussian distributions. Particle filters provide a sample-based method of representing densities that removes that restriction and also is tolerant of occasional large deviations from the well-behaved model. In this assignment, you will be implementing a particle filter to track entities in video.

Three video (.avi) files are provided in the input directory:
- pres_debate.avi, which shows two candidates in a 2012 town hall debate,
- noisy_debate.avi, which is the same video overlaid with fluctuating Gaussian noise, and,
- pedestrians.avi, which shows a group of people crossing a street in London.

For each video file, there is a text file that contains the object bounding box at the first frame that you can use for initialization. 
The format of the text file is:
- x y
- w h

where (x, y) is the top-left coordinate (not center), and (w, h) is the size (width, height) of the bounding box.


## Class Notes: Algorithm Sketch and Parameters

1.  Start off with a randomized set of particles and normalization constant of zero (constant weights).  The particles in our example represent the center of an image patch that we are attempting to track through time.

Then, for each particle in the new sample:

2. Choose one sample with a probabilty defined by its weight from previous time step.
3. Assume the state in the current time is a function of the state in the previous time period and some dynamics model (in our case, just xy Gaussian noise). Obtain a measurement (in our case, the MSE difference between the template of the face and the template centered about the sample). Using this measurement, obtain a probability of obtaining such a measurement and reweight the sample accordingly.
4. Place the sample and its weight back into a new set of particles.
5. Once the specified number of samples is obtained in the above manner, adjust all weights in the new set of particles so that they sum to 1.
6. Resample from the new set, and return to the loop starting at step (2).

For this specific problem set:

1.  The "Model" we are tracking is an image patch.
2.  The "State" is xy-position of this window in the scene.
3.  The "Dynamics model" is Gaussian noise in xy coordinate system.
4.  Our "Measurement" is the MSE between the sampled state and the previous or base state.
5.  Our "Sensor model" returns a probability of observing our new state given the previous belief about the state, based on 4.

## Question 1. Particle Filter Tracking - The Presidential Debate

In [5]:
# libraries and header
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

%matplotlib inline

# file and directory information
os.chdir('/Users/justinfung/Desktop/udacity/cv/hw/ps6_python/input')
movie1 = 'pres_debate.mov'
movie2 = 'noisy_debate.mov'

# parameters for the particle filter
num_particles = 200
sigma_MSE = 5
sigma_Gauss = 10
alpha = 0

# initial detection window, where (u,v,,) is the top-left coordinate (not center), 
# and (,,m,n) is the size (width, height) of the bounding box.
template = (320,175,103,129) # problem 1.a
#template = (350,210,40,60) # problem 1.b
#template = (300,150,140,180) # problem 1.b

# start a videowriter object for output
# from: http://www.pyimagesearch.com/2016/02/22/writing-to-video-with-opencv/
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mov',fourcc, 30.0, (1280,720))

### Visualization Methods

In order to visualize the tracker’s behavior you will need to overlay each successive frame with the following visualizations:
- Every particle’s (u,v) location in the distribution should be plotted by drawing a colored dot point on the image. Remember that this should be the center of the window, not the corner.
- Draw the rectangle of the tracking window associated with the Bayesian estimate for the current location which is simply the weighted mean of the (u,v) of the particles.
- Finally we need to get some sense of the standard deviation or spread of the distribution. First, find the distance of every particle to the weighted mean. Next, take the weighted sum of these distances and plot a circle centered at the weighted mean with this radius.

In [6]:
def draw_centerpoints(img,num_points,pointsarr,radius,color,offsetx,offsety):
    """
    Draws a colored dot representing every particle’s (u,v) location in the distribution. 
    This dot represents the center of the window, not the corner.
    ----
    img: frame from video on which to overlay particles as points
    num_points: number of points to draw, should be same as num of particles
    pointsarr: 2D array of points (shape ex: (2,200)) in which first column is row, 
               2nd is col; points indicate TOP LEFT extent of window
    radius: radius of the visualized point
    color: color in BGR as a tuple
    offsetx: distance to center of detection window from left
    offsety: distnce to center of detection window from top
    """
    for i in xrange(num_points):
        cv2.circle(img,
                   (pointsarr[1][i]+offsetx,pointsarr[0][i]+offsety),
                   radius, 
                   color, 
                   -1)

def draw_window(img, topleftpoint, radius, color, offsetx, offsety):
    """
    Draws the rectangle of the tracking window associated with the Bayesian estimate 
    for the current location which is simply the weighted mean of the (u,v) of the particles.
    -----
    img: frame from video on which to overlay particles as points
    topleftpoint: xy val of top left point of window as a tuple
    radius: radius of the visualized window
    color: color in BGR as a tuple
    offsetx: distance to center of detection window from left
    offsety: distnce to center of detection window from top
    """    
    cv2.rectangle(img,
                  (topleftpoint[0],topleftpoint[1]),
                  (topleftpoint[0]+offsetx*2, topleftpoint[1]+offsety*2),
                  color,
                  3)

def draw_circle(img, pointsarr, weighted_mean, weights, color, offsetx, offsety):
    """
    Draws a circle centered at the weighted mean of the particles with a radius that
    represents the standard deviation or spread of the distribution.
    -----
    img: frame from video on which to overlay particles as points
    pointsarr: 2D array of points (shape ex: (2,200)) in which first column is row, 
               2nd is col; points indicate TOP LEFT extent of window
    weighted_mean: weight_mean as tuple of (x,y)
    weights: weights of the distribution of particles
    color: color in BGR as a tuple
    offsetx: distance to center of detection window from left
    offsety: distnce to center of detection window from top
    """
    distances = np.zeros(pointsarr.shape[1])
    i = 0
    for i in xrange(distances.shape[0]):
        distances[i] = ((pointsarr[1][i]-weighted_mean[0])**2 + (pointsarr[0][i]-weighted_mean[1])**2) ^ (1/2)
    weighted_sum = int(np.dot(distances,weights)/100)
    
    cv2.circle(img,
               (weighted_mean[0] + offsetx, weighted_mean[1] + offsety), 
               weighted_sum, 
               (0,0,255), 
               -1)

### Initialize Particle Filter

In [7]:
# read in video and get meta data
# take first non-empty frame of the video
cap = cv2.VideoCapture(movie2)
start = 0
ret, initframe = cap.read()
while initframe.mean() == 0:
    ret, initframe = cap.read()
    start += 1
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

# initialize arrays to hold particles and normalization constants
uv = np.zeros((num_frames*2*num_particles),dtype=np.int).reshape(num_frames,2,num_particles)
eta = np.zeros((num_frames*num_particles),dtype=np.float).reshape(num_frames,num_particles)

# setup initial location of window and the (1) state,
# using the following notation:
u = template[0] # u is top left x
v = template[1] # v is top left y
m = template[2] # m is width of window (columns)
n = template[3] # n is height of window (rows)
u_p = u + m/2  # u_p is center x
v_p = v + n/2  # v_p is center y
template_patch = initframe[v:v+n,u:u+m,:] # img[rows,cols,channels]
template_center = initframe[v_p,u_p,:]

# initialize top-left particle coordinates and weights
uv[0][0] = np.random.randint(v-n/2,v+n/2,size = num_particles) # random y-coordinates (row)
uv[0][1] = np.random.randint(u-m/2,u+m/2,size = num_particles) # random x-coordinates (column)
eta[0] = np.ones(num_particles) / num_particles # normalized constant weights

In [8]:
# MAIN
# -> loop through all the frames
for fr in xrange(start,num_frames):
    ret,frame = cap.read()
    eta_sum = 0
    
    for particle in xrange(num_particles):
        # resample from previous particle states
        s_idx = np.random.choice(num_particles, 1, p=eta[fr-1]).item() #random choice by weight
        
        # dynamics model from (3)        
        uv[fr][0][particle] = uv[fr-1][0][s_idx] + np.round(np.random.normal(0,sigma_Gauss))
        uv[fr][1][particle] = uv[fr-1][1][s_idx] + np.round(np.random.normal(0,sigma_Gauss))
        
        # check for coordinates outside the extent of the frame and adjust if necessary
        if uv[fr][1][particle] < 0:
            uv[fr][1][particle] = 0
        elif uv[fr][1][particle] > frame_width:
            uv[fr][1][particle] = frame_width
        
        if uv[fr][0][particle] < 0:
            uv[fr][0][particle] = 0
        elif uv[fr][0][particle] > frame_height:
            uv[fr][0][particle] = frame_height
        
        # measurement from (4)
        u_p = uv[fr][1][particle] + m/2; #center x-coor/col
        v_p = uv[fr][0][particle] + n/2; #center y-coor/row
        image_patch = frame[uv[fr][0][particle]:uv[fr][0][particle] + n,
                            uv[fr][1][particle]:uv[fr][1][particle] + m, :]
        particleMSE = 1. / (m*n) * np.sum((template_patch - image_patch)**2)

        # probability of observing such a state from (5)
        eta[fr][particle] = np.exp(-(particleMSE / (2 * sigma_MSE ^ 2)))
        
        # track a cumsum for normalization
        eta_sum += eta[fr][particle]
    
    # normalize probabilities
    eta[fr] = eta[fr] / eta_sum

    # visualizations for display
    draw_centerpoints(frame, num_particles, uv[fr], 3, (0,255,0), m/2, n/2)
    
    weighted_mean_x = int(np.average(uv[fr][1], weights=eta[fr]))
    weighted_mean_y = int(np.average(uv[fr][0], weights=eta[fr]))
    weighted_mean = (weighted_mean_x,weighted_mean_y)
    
    draw_window(frame, weighted_mean, 3, (0,255,0), m/2, n/2)
    draw_circle(frame, uv[fr], weighted_mean, eta[fr],(0,0,255), m/2, n/2)
    
    out.write(frame)
    
    # print status update
    if fr % 10 == 0:
        print fr , "frames complete."
        
    # capture some frames for output
    if fr == 28:
        ps6_1_a_2 = frame
    elif fr == 84:
        ps6_1_a_3 = frame
    elif fr == 144: 
        ps6_1_a_4 = frame
    elif fr == 14:
        ps6_1_e_1 = frame
    elif fr == 32:
        ps6_1_e_2 = frame
    elif fr == 46:
        ps6_1_e_3 = frame

# cleanup resources
cap.release()
out.release()
cv2.destroyAllWindows()

10 frames complete.
20 frames complete.
30 frames complete.
40 frames complete.
50 frames complete.
60 frames complete.
70 frames complete.
80 frames complete.
90 frames complete.
100 frames complete.
110 frames complete.
120 frames complete.
130 frames complete.
140 frames complete.
150 frames complete.
160 frames complete.
170 frames complete.
180 frames complete.
190 frames complete.
200 frames complete.
210 frames complete.
220 frames complete.
230 frames complete.
240 frames complete.
250 frames complete.
260 frames complete.
270 frames complete.
280 frames complete.
290 frames complete.
300 frames complete.
310 frames complete.
320 frames complete.
330 frames complete.
340 frames complete.
350 frames complete.
360 frames complete.
370 frames complete.
380 frames complete.
390 frames complete.
400 frames complete.
410 frames complete.
420 frames complete.
430 frames complete.
440 frames complete.
450 frames complete.
460 frames complete.
470 frames complete.
480 frames complete.
4

## Question 1: Ouput

#### 1A. Implement the particle filter and run it on the pres debate.avi clip. 

> You should begin by attempting to track Romney’s face. Tweak the parameters including window size until you can get the tracker to follow his face faithfully (5-15 pixels) up until he turns his face significantly. Run the tracker and save the video frames 28, 84, and 144 with the visualizations overlaid.

In [ ]:
#Output:
os.chdir('/Users/justinfung/Desktop/udacity/cv/hw/ps6_python/output')

#the image patch used for tracking as ps6-1-a-1.png
cv2.imwrite("ps6-1-a-1.png",template_patch)

#the image frame number 28 with overlaid visualizations as ps6-1-a-2.png
cv2.imwrite("ps6-1-a-2.png",ps6_1_a_2)

#the image frame number 84 with overlaid visualizations as ps6-1-a-3.png
cv2.imwrite("ps6-1-a-3.png",ps6_1_a_3)

#the image frame number 144 with overlaid visualizations as ps6-1-a-4.png
cv2.imwrite("ps6-1-a-4.png",ps6_1_a_4)


#### 1B. Experiment with different dimensions for the window image patch you are trying to track. 

>Decrease the window size until the performance of the tracker degrades significantly. Try significantly larger windows than what worked in 1-a. Discuss the trade-offs of window size and what makes some image patches work better than others for tracking.

>Describe 2-3 advantages of larger window size and 2-3 advantages of smaller window size:

* __Larger Window Size__:
  - Theoretically less susceptible to noise in the frame due to larger inference region in the measurement step.
  - Should be more robust to suboptimal convergence due to more granularity in a larger window.


* __Smaller Window Size__:
  - Computationally more-efficient as computations increase in O(mn) with window diameter.
  - Should be faster to convergence as MSE has a greater variance with less number of pixels.


#### 1C. Adjust the σMSE parameter to higher and lower values and run the tracker.

>Discuss how changing σMSE parameter alters the results and attempt to explain why.

* __Larger σMSE parameter__:
  - We increase our parameter by a factor of 2, to a value of 20.
  - Mathematically, a higher parameter widens the spread of a Gaussian distribution, which in effect penalizes dissimilarity to a lesser degree and therefore encourages "exploration" of the particles in the sensor model due to higher weights being placed on outlying measurements..
  - In our problem, this causes the tracker to drift significantly under occlusion.  However, due to the greater value of the parameter, we witness our tracker regain confident tracking when the candidate's face becomes visible again.
  

* __Smaller σMSE parameter__:
  - We reduce our parameter by a factor of 2, to a value of 5.
  - Mathematically, this penalizes dissimilarity to a greater degree and therefore reduces "exploration" of the particles in the sensor model.
  - In our problem, this causes the tracker to faithfully stay centered on the presidential candidates face even in the case of occlusion.  The tracker appears to move very little in any given frame.


#### 1D. Try and optimize the number of particles needed to track the target.

* __Optimized particle number:__
  - We choose the range of 100-200 particles to be "optimal", though "optimality" in our context is not an entirely measurable thing.  We consider any tracker to recover properly from a servere occlusion to be acceptable, and therefore we choose the lowest range of particle for which we observe this behavior.


* __Discuss the trade-offs of using a larger number of particles to represent the distribution:__
  - Computational requirements scale linearly with the addition of more particles, so all things equal, we prefer less particles.
  - Less particles however may subject us to mischaracterization of the underlying dynamics due to undersampling- in this case we might observe our tracker drift into an unrecoverable local maximum.
  - Larger particle numbers gives us greater confidence in our underlying dynamics and will leave us less susceptible to local maxima.


#### 1E. Run your tracker on noisy debate.avi and see what happens. 

>Tune your parameters so that the cluster is able to latch back onto his face after the noise disappears. Include varying σMSE. Report how the particles respond to increasing and decreasing noise. Save the video frames 14, 32, and 46 with the visualizations overlaid.

* __Discuss how you managed to tune the parameters:__
  - The pulsing of the red circle (the indicator of spread of particles) and the spread of the green particles themselves is highly correlated with the addition of noise.  This is too be expected as the addition of random noise will have a direct negative impact on measured MSE and thus the updated weights of particles in each frame.
  - The MSE was lowered so as not to encourage too much drift of the tracker that might be induced by the addition of noise.
  - The same number of particles (n = 100) was utilized as in the denoised case.
  - Under these conditions, we are still able to obtain acceptable tracking of the candidates face.


In [ ]:
#Output:
os.chdir('/Users/justinfung/Desktop/udacity/cv/hw/ps6_python/output')

# the image frame number 14 with overlaid visualizations as ps6-1-e-1.png
cv2.imwrite("ps6-1-e-1.png",ps6_1_e_1)

# the image frame number 32 with overlaid visualizations as ps6-1-e-2.png
cv2.imwrite("ps6-1-e-2.png",ps6_1_e_2)

# the image frame number 46 with overlaid visualizations as ps6-1-e-3.png
cv2.imwrite("ps6-1-e-3.png",ps6_1_e_3)

## Question 2: Appearance Model Update

> Let’s say we’d now like to track Romney’s left hand (the one not holding the mic). You might find that it’s difficult to keep up using the naive tracker you wrote in question 1. The issue is that while making rapid hand gestures, the appearance of the hand significantly changes as it rotates and changes perspective. However, if we make the assumption that the appearance changes smoothly over time, we can update our appearance model over time.

>Modify your existing tracker to include a step which uses the history to update the tracking window model. We can accomplish this using what’s called an Infinite Impulse Response (IIR) filter. The concept is simple: we first find the best tracking window for the current particle distribution as displayed in the visualizations. Then we just update the current window model to be a weighted sum of the last model and the current best estimate:

> __Template(t) = (alpha) Best(t) + (1 - alpha) Template(1-t)__,

> where Best(t) is the patch of the best estimate or mean estimate. It’s easy to see that by recursively updating this sum, the window implements an exponentially decaying weighted sum of (all) the past windows.

#### 2A. Implement the appearance model update. 

> Run the tracker on pres_debate.avi and adjust parameters until you can track Romney’s hand up to frame 140. Run the tracker and save the video frames 15, 50, and 140 with the visualizations overlaid.

In [17]:
# file and directory information
os.chdir('/Users/justinfung/Desktop/udacity/cv/hw/ps6_python/input')
movie1 = 'pres_debate.mov'

# parameters for the particle filter
num_particles = 200
sigma_MSE = 20
sigma_Gauss = 10
alpha = 0

# initial detection window, where (u,v,,) is the top-left coordinate (not center), 
# and (,,m,n) is the size (width, height) of the bounding box.
template = (510,380,125,125) # problem 2.a

# start a videowriter object for output
# from: http://www.pyimagesearch.com/2016/02/22/writing-to-video-with-opencv/
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_hand.mov',fourcc, 30.0, (1280,720))

In [18]:
# read in video and get meta data
# take first non-empty frame of the video
cap = cv2.VideoCapture(movie1)
start = 0
ret, initframe = cap.read()
while initframe.mean() == 0:
    ret, initframe = cap.read()
    start += 1
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

# initialize arrays to hold particles and normalization constants
uv = np.zeros((num_frames*2*num_particles),dtype=np.int).reshape(num_frames,2,num_particles)
eta = np.zeros((num_frames*num_particles),dtype=np.float).reshape(num_frames,num_particles)

# setup initial location of window and the (1) state,
# using the following notation:
u = template[0] # u is top left x
v = template[1] # v is top left y
m = template[2] # m is width of window (columns)
n = template[3] # n is height of window (rows)
u_p = u + m/2  # u_p is center x
v_p = v + n/2  # v_p is center y
template_patch = initframe[v:v+n,u:u+m,:] # img[rows,cols,channels]
template_center = initframe[v_p,u_p,:]

# initialize top-left particle coordinates and weights
uv[0][0] = np.random.randint(v-n/2,v+n/2,size = num_particles) # random y-coordinates (row)
uv[0][1] = np.random.randint(u-m/2,u+m/2,size = num_particles) # random x-coordinates (column)
eta[0] = np.ones(num_particles) / num_particles # normalized constant weights

In [19]:
# MAIN_2
# -> loop through all the frames
for fr in xrange(start,num_frames):
    ret,frame = cap.read()
    eta_sum = 0
    
    for particle in xrange(num_particles):
        # resample from previous particle states
        s_idx = np.random.choice(num_particles, 1, p=eta[fr-1]).item() #random choice by weight
        
        # dynamics model from (3)        
        uv[fr][0][particle] = uv[fr-1][0][s_idx] + np.round(np.random.normal(0,sigma_Gauss))
        uv[fr][1][particle] = uv[fr-1][1][s_idx] + np.round(np.random.normal(0,sigma_Gauss))
        
        # check for coordinates outside the extent of the frame and adjust if necessary
        if uv[fr][1][particle] < 0:
            uv[fr][1][particle] = 0
        elif uv[fr][1][particle] > frame_width:
            uv[fr][1][particle] = frame_width
        
        if uv[fr][0][particle] < 0:
            uv[fr][0][particle] = 0
        elif uv[fr][0][particle] > frame_height:
            uv[fr][0][particle] = frame_height
        
        # measurement from (4)
        u_p = uv[fr][1][particle] + m/2; #center x-coor/col
        v_p = uv[fr][0][particle] + n/2; #center y-coor/row
        image_patch = frame[uv[fr][0][particle]:uv[fr][0][particle] + n,
                            uv[fr][1][particle]:uv[fr][1][particle] + m, :]
        particleMSE = 1. / (m*n) * np.sum((template_patch - image_patch)**2)

        # probability of observing such a state from (5)
        eta[fr][particle] = np.exp(-(particleMSE / (2 * sigma_MSE ^ 2)))
        
        # track a cumsum for normalization
        eta_sum += eta[fr][particle]
    
    # normalize probabilities
    eta[fr] = eta[fr] / eta_sum

    # visualizations for display
    draw_centerpoints(frame, num_particles, uv[fr], 3, (0,255,0), m/2, n/2)
    
    weighted_mean_x = int(np.average(uv[fr][1], weights=eta[fr]))
    weighted_mean_y = int(np.average(uv[fr][0], weights=eta[fr]))
    weighted_mean = (weighted_mean_x,weighted_mean_y)
    
    draw_window(frame, weighted_mean, 3, (0,255,0), m/2, n/2)
    draw_circle(frame, uv[fr], weighted_mean, eta[fr],(0,0,255), m/2, n/2)
    
    out.write(frame)
    
    # print status update
    if fr % 10 == 0:
        print fr , "frames complete."
        
    # capture some frames for output
    if fr == 28:
        ps6_1_a_2 = frame
    elif fr == 84:
        ps6_1_a_3 = frame
    elif fr == 144: 
        ps6_1_a_4 = frame
    elif fr == 14:
        ps6_1_e_1 = frame
    elif fr == 32:
        ps6_1_e_2 = frame
    elif fr == 46:
        ps6_1_e_3 = frame

# cleanup resources
cap.release()
out.release()
cv2.destroyAllWindows()

10 frames complete.
20 frames complete.
30 frames complete.
40 frames complete.
50 frames complete.
60 frames complete.
70 frames complete.
80 frames complete.
90 frames complete.
100 frames complete.
110 frames complete.
120 frames complete.
130 frames complete.
140 frames complete.
150 frames complete.
160 frames complete.
170 frames complete.
180 frames complete.
190 frames complete.
200 frames complete.
210 frames complete.
220 frames complete.
230 frames complete.
240 frames complete.
250 frames complete.
260 frames complete.
270 frames complete.
280 frames complete.
290 frames complete.
300 frames complete.
310 frames complete.
